## Get statements about regulation of cell motility from INDRA DB

In [1]:
# INDRA can be installed with `pip install indra`
# GILDA can be installed with `pip install gilda`

# If you want to run this, let us know and we will set you up an API key

import csv
import json
from copy import deepcopy
from tqdm import tqdm
from indra.preassembler import Preassembler # For combining duplicate statements
from indra.sources import indra_db_rest as idr # For querying the INDRA DB REST API
from indra.tools import assemble_corpus as ac # Useful assembly and filtering functions
from indra.assemblers.html import HtmlAssembler # Rendering stmts as HTML
from indra.util import statement_presentation as sp # Rendering stmts as HTML
from indra.preassembler.grounding_mapper import gilda
from indra.statements import stmts_to_json, Activation, Inhibition


INFO: [2021-01-15 09:49:55] /Users/johnbachman/Dropbox/1johndata/Knowledge File/Biology/Research/Big Mechanism/adeft/adeft/recognize.py - OneShotRecognizer not available. AdeftLongformScorer has not been built successfully.

(Background on this error at: http://sqlalche.me/e/e3q8)


In [2]:
# List of motility target terms
motility_terms = ['GO:0048870@GO', # GO Cell Motility
                  'GO:0016477@GO'] # GO Cell migration (synonymous with MESH D002465)

In [3]:
# Get all statements for regulators of the various motility terms
last_proc = None
for motility_term in motility_terms:
    # Returns INDRA DB REST Processor object with all textual evidences
    idrp = idr.get_statements(object=motility_term, ev_limit=10)
    # Merge results information with results for previous term
    if last_proc is not None:
        idrp.merge_results(last_proc)
    last_proc = idrp

INFO: [2021-01-15 09:50:03] indra.sources.indra_db_rest.util - query: https://db.indra.bio/latest/statements/from_agents?object=GO:0048870@GO&offset=0
INFO: [2021-01-15 09:50:03] indra.sources.indra_db_rest.util - params: {'ev_limit': 10, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-01-15 09:50:07] indra.sources.indra_db_rest.util - query: https://db.indra.bio/latest/statements/from_agents?object=GO:0048870@GO&offset=500
INFO: [2021-01-15 09:50:07] indra.sources.indra_db_rest.util - params: {'ev_limit': 10, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-01-15 09:50:09] indra.sources.indra_db_rest.util - query: https://db.indra.bio/latest/statements/from_agents?object=GO:0048870@GO&offset=1000
INFO: [2021-01-15 09:50:09] indra.sources.indra_db_rest.util - params: {'ev_limit': 10, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-01-15 09:50:11] indra.sources.indra_db_rest.util - query: https://db.indra.bio/latest/statements/from_agents?object=GO:0048870@GO&offs

INFO: [2021-01-15 09:51:08] indra.sources.indra_db_rest.util - query: https://db.indra.bio/latest/statements/from_agents?object=GO:0016477@GO&offset=9000
INFO: [2021-01-15 09:51:08] indra.sources.indra_db_rest.util - params: {'ev_limit': 10, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-01-15 09:51:09] indra.sources.indra_db_rest.util - query: https://db.indra.bio/latest/statements/from_agents?object=GO:0016477@GO&offset=9500
INFO: [2021-01-15 09:51:09] indra.sources.indra_db_rest.util - params: {'ev_limit': 10, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-01-15 09:51:11] indra.sources.indra_db_rest.util - query: https://db.indra.bio/latest/statements/from_agents?object=GO:0016477@GO&offset=10000
INFO: [2021-01-15 09:51:11] indra.sources.indra_db_rest.util - params: {'ev_limit': 10, 'best_first': True, 'api_key': '[api-key]'}
INFO: [2021-01-15 09:51:12] indra.sources.indra_db_rest.util - query: https://db.indra.bio/latest/statements/from_agents?object=GO:0016477@GO

Let's see how many statements we got

In [4]:
len(idrp.statements)

23754

Because we'll need it later, we build a mapping from the statement UUID to the source and evidence counts. The UUID doesn't change when we re-ground the statements, whereas the statement hash does, making it possible for us to retrieve the evidence counts when generating the tabular and HTML results.

In [5]:
ev_counts_by_hash = idrp.get_ev_counts()
ev_counts_by_uuid = {s.uuid: ev_counts_by_hash[s.get_hash()] for s in idrp.statements}
src_counts_by_hash = idrp.get_source_counts()
src_counts_by_uuid = {s.uuid: src_counts_by_hash[s.get_hash()] for s in idrp.statements}

Now make a copy of the statements and fix (in place, in the copy) a few cases of statements missing standard identifiers by grounding to IDs with Gilda (see http://grounding.indra.bio)

In [6]:
grounded_stmts = deepcopy(idrp.statements)

Now use GILDA to find standard identifiers for a subset of statements that the source reading systems couldn't normalize.

In [7]:
grounded_stmts = gilda.ground_statements(grounded_stmts, mode='local', ungrounded_only=True)

INFO: [2021-01-15 09:52:04] gilda.grounder - Looking up the following strings: sirna
INFO: [2021-01-15 09:52:04] gilda.grounder - Comparing siRNA with 1 entries
INFO: [2021-01-15 09:52:04] gilda.grounder - Running disambiguation for siRNA
INFO: [2021-01-15 09:52:04] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /Users/johnbachman/.indra/bio_ontology/1.7/bio_ontology.pkl
INFO: [2021-01-15 09:52:14] gilda.grounder - Looking up the following strings: sdf1α, sdf 1alpha, sdf1a, sdf1alpha
INFO: [2021-01-15 09:52:14] gilda.grounder - Comparing SDF-1alpha with 2 entries
INFO: [2021-01-15 09:52:14] gilda.grounder - Running disambiguation for SDF-1alpha
INFO: [2021-01-15 09:52:14] gilda.grounder - Looking up the following strings: laminin
INFO: [2021-01-15 09:52:14] gilda.grounder - Comparing laminin with 1 entries
INFO: [2021-01-15 09:52:14] gilda.grounder - Running disambiguation for laminin
INFO: [2021-01-15 09:52:14] gilda.grounder - Looking up the following strings:

INFO: [2021-01-15 09:52:14] gilda.grounder - Looking up the following strings: ptpmu, ptpμ
INFO: [2021-01-15 09:52:14] gilda.grounder - Comparing PTPmu with 0 entries
INFO: [2021-01-15 09:52:14] gilda.grounder - Looking up the following strings: rantes
INFO: [2021-01-15 09:52:14] gilda.grounder - Comparing RANTES with 1 entries
INFO: [2021-01-15 09:52:14] gilda.grounder - Running disambiguation for RANTES
INFO: [2021-01-15 09:52:14] gilda.grounder - Looking up the following strings: uca1
INFO: [2021-01-15 09:52:14] gilda.grounder - Comparing UCA1 with 1 entries
INFO: [2021-01-15 09:52:14] gilda.grounder - Running disambiguation for UCA1
INFO: [2021-01-15 09:52:14] gilda.grounder - Looking up the following strings: mir 125b, mir125b
INFO: [2021-01-15 09:52:14] gilda.grounder - Comparing miR-125b with 0 entries
INFO: [2021-01-15 09:52:14] gilda.grounder - Looking up the following strings: mir143, mir 143
INFO: [2021-01-15 09:52:14] gilda.grounder - Comparing miR-143 with 1 entries
INFO: 

INFO: [2021-01-15 09:52:14] gilda.grounder - Looking up the following strings: mir223, mir 223
INFO: [2021-01-15 09:52:14] gilda.grounder - Comparing miR-223 with 1 entries
INFO: [2021-01-15 09:52:14] gilda.grounder - Running disambiguation for miR-223
INFO: [2021-01-15 09:52:14] gilda.grounder - Looking up the following strings: mir 200b, mir200b
INFO: [2021-01-15 09:52:14] gilda.grounder - Comparing miR-200b with 1 entries
INFO: [2021-01-15 09:52:14] gilda.grounder - Running disambiguation for miR-200b
INFO: [2021-01-15 09:52:14] gilda.grounder - Looking up the following strings: emt
INFO: [2021-01-15 09:52:14] gilda.grounder - Comparing EMT with 3 entries
INFO: [2021-01-15 09:52:14] gilda.grounder - Running disambiguation for EMT
INFO: [2021-01-15 09:52:14] gilda.grounder - Result from Adeft: {'HGNC:6179': 3.190383253599192e-06, 'MESH:D058750': 0.9998914999797809, 'ungrounded': 0.00010530963696547337}
INFO: [2021-01-15 09:52:14] gilda.grounder - Multiplying the score of "epithelial 

INFO: [2021-01-15 09:52:14] gilda.grounder - Comparing miR-183 with 1 entries
INFO: [2021-01-15 09:52:14] gilda.grounder - Running disambiguation for miR-183
INFO: [2021-01-15 09:52:14] gilda.grounder - Looking up the following strings: nsc23766
INFO: [2021-01-15 09:52:14] gilda.grounder - Comparing NSC23766 with 2 entries
INFO: [2021-01-15 09:52:14] gilda.grounder - Running disambiguation for NSC23766
INFO: [2021-01-15 09:52:14] gilda.grounder - Looking up the following strings: cbp501
INFO: [2021-01-15 09:52:14] gilda.grounder - Comparing CBP501 with 0 entries
INFO: [2021-01-15 09:52:14] gilda.grounder - Looking up the following strings: mir 24 3p, mir243p
INFO: [2021-01-15 09:52:14] gilda.grounder - Comparing miR-24-3p with 0 entries
INFO: [2021-01-15 09:52:14] gilda.grounder - Looking up the following strings: mir 23b, mir23b
INFO: [2021-01-15 09:52:14] gilda.grounder - Comparing miR-23b with 1 entries
INFO: [2021-01-15 09:52:14] gilda.grounder - Running disambiguation for miR-23b


INFO: [2021-01-15 09:52:14] gilda.grounder - Multiplying the score of "cytidine 5'-monophosphate" with 0.978
INFO: [2021-01-15 09:52:14] gilda.grounder - Multiplying the score of "phosphatidylcholine(1+)" with 0.978
INFO: [2021-01-15 09:52:14] gilda.grounder - Multiplying the score of "polycarbonate macromolecule" with 0.978
INFO: [2021-01-15 09:52:14] gilda.grounder - Multiplying the score of "phosphocholine group" with 0.978
INFO: [2021-01-15 09:52:14] gilda.grounder - Multiplying the score of "polycarbonate polymer" with 0.978
INFO: [2021-01-15 09:52:14] gilda.grounder - Multiplying the score of "PC" with 0.001
INFO: [2021-01-15 09:52:14] gilda.grounder - Multiplying the score of "PODXL" with 0.978
INFO: [2021-01-15 09:52:14] gilda.grounder - Looking up the following strings: laminin
INFO: [2021-01-15 09:52:14] gilda.grounder - Comparing Laminin with 1 entries
INFO: [2021-01-15 09:52:14] gilda.grounder - Running disambiguation for Laminin
INFO: [2021-01-15 09:52:14] gilda.grounder -

...

INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing Cripto-1 with 1 entries
INFO: [2021-01-15 09:52:34] gilda.grounder - Running disambiguation for Cripto-1
INFO: [2021-01-15 09:52:34] gilda.grounder - Looking up the following strings: robo receptors, robo receptor
INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing Robo receptors with 0 entries
INFO: [2021-01-15 09:52:34] gilda.grounder - Looking up the following strings: 4atwist1, 4a twist1
INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing 4A-TWIST1 with 0 entries
INFO: [2021-01-15 09:52:34] gilda.grounder - Looking up the following strings: shiga toξnencoding bacteriophage, shiga toxin encoding bacteriophage, shiga toxinencoding bacteriophage
INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing Shiga toxin-encoding bacteriophage with 0 entries
INFO: [2021-01-15 09:52:34] gilda.grounder - Looking up the following strings: pugnac
INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing PUGNAc with 0 entries
INFO: [2021-01-1

INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing 3F3A mAb with 0 entries
INFO: [2021-01-15 09:52:34] gilda.grounder - Looking up the following strings: egfl
INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing EGFL with 0 entries
INFO: [2021-01-15 09:52:34] gilda.grounder - Looking up the following strings: 1 5microm tdb, 15microm tdb
INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing 1-5microM TDB with 0 entries
INFO: [2021-01-15 09:52:34] gilda.grounder - Looking up the following strings: formyl peptide receptor 1
INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing Formyl peptide receptor 1 with 1 entries
INFO: [2021-01-15 09:52:34] gilda.grounder - Running disambiguation for Formyl peptide receptor 1
INFO: [2021-01-15 09:52:34] gilda.grounder - Looking up the following strings: metalloproteases, mηlloproteases, metalloproteas
INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing metalloproteases with 1 entries
INFO: [2021-01-15 09:52:34] gilda.grounder - Running disa

INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing csrA with 0 entries
INFO: [2021-01-15 09:52:34] gilda.grounder - Looking up the following strings: pct
INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing PCT with 0 entries
INFO: [2021-01-15 09:52:34] gilda.grounder - Looking up the following strings: integrin a, integrin α, integrin alpha
INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing Integrin alpha with 0 entries
INFO: [2021-01-15 09:52:34] gilda.grounder - Looking up the following strings: cd11d
INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing CD11d with 1 entries
INFO: [2021-01-15 09:52:34] gilda.grounder - Running disambiguation for CD11d
INFO: [2021-01-15 09:52:34] gilda.grounder - Looking up the following strings: abl inhibitor, abl inhibitors
INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing Abl inhibitors with 2 entries
INFO: [2021-01-15 09:52:34] gilda.grounder - Running disambiguation for Abl inhibitors
INFO: [2021-01-15 09:52:34] gilda.grounder - 

INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing GO6983 with 0 entries
INFO: [2021-01-15 09:52:34] gilda.grounder - Looking up the following strings: adscs cm, adscscm
INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing ADSCs-CM with 0 entries
INFO: [2021-01-15 09:52:34] gilda.grounder - Looking up the following strings: raptor or rictor
INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing raptor or rictor with 0 entries
INFO: [2021-01-15 09:52:34] gilda.grounder - Looking up the following strings: rmasπn, rmaspin
INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing rMaspin with 0 entries
INFO: [2021-01-15 09:52:34] gilda.grounder - Looking up the following strings: dbl1, dbl 1
INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing Dbl-1 with 0 entries
INFO: [2021-01-15 09:52:34] gilda.grounder - Looking up the following strings: proncadherin, pron cadherin
INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing proN-cadherin with 0 entries
INFO: [2021-01-15 09:52:34] gilda

INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing EPHA2 receptor with 1 entries
INFO: [2021-01-15 09:52:34] gilda.grounder - Running disambiguation for EPHA2 receptor
INFO: [2021-01-15 09:52:34] gilda.grounder - Looking up the following strings: 15lipoxygenase1, 15 lipoxygenase 1
INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing 15-lipoxygenase-1 with 0 entries
INFO: [2021-01-15 09:52:34] gilda.grounder - Looking up the following strings: beta1 and alpha5integrins, b1 and a5integrins, beta1 and alpha5integrin, beta1 and alpha5 integrins, β1 and α5integrins
INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing beta1- and alpha5-integrins with 0 entries
INFO: [2021-01-15 09:52:34] gilda.grounder - Looking up the following strings: mp
INFO: [2021-01-15 09:52:34] gilda.grounder - Comparing MP with 3 entries
INFO: [2021-01-15 09:52:34] gilda.grounder - Running disambiguation for MP
INFO: [2021-01-15 09:52:34] gilda.grounder - Looking up the following strings: jail
INFO: [2021-01-

## Filter to statements involving kinases

Load the kinase list from the TSV:

In [8]:
# List of kinases from Uniprot
with open('kinases.tsv', 'r') as f:
    csvreader = csv.reader(f, delimiter='\t')
    next(csvreader) # Skip the header row
    kinases = [gene_name for up_id, gene_name, species, up_name
                         in csvreader]

Let's take a look at processed the kinase list--could use other kinase lists as well.

In [9]:
kinases[:10]

['AAK1',
 'PRKAA1',
 'PRKAA2',
 'ABL1',
 'ABL2',
 'ACAD10',
 'ACAD11',
 'TNK2',
 'ACVR1B',
 'ACVR1C']

How many kinases?

In [10]:
len(kinases)

539

Filter to statements containing at least one kinase (in this case the subject), and allow families that have the genes as members, e.g. the families "ERK", "AKT", etc. (these are denoted by FamPlex identifiers)

In [11]:
kin_stmts = ac.filter_gene_list(grounded_stmts, kinases, 'one', allow_families=True, remove_bound=True)

INFO: [2021-01-15 09:52:35] indra.tools.assemble_corpus - Filtering 23754 statements for ones containing "one" of: AAK1, PRKAA1, PRKAA2, ABL1, ABL2, ACAD10, ACAD11, TNK2, ACVR1B, ACVR1C, ACVRL1, ACVR1, ADCK1, ADCK2, ADCK5, AKT1, AKT2, AKT3, ALK, ALPK1, ALPK2, ALPK3, AMHR2, ANKK1, NPR1, NPR2, ARAF, GRK2, GRK3, ATM, ATR, AURKA, AURKB, AURKC, ACVR2A, ACVR2B, BLK, BMP2K, BMPR2, BMPR1A, BMPR1B, BMX, BRAF, BRSK1, BRSK2, BTK, BUB1, BUB1B, CAMKV, CDK11A, CDK11B, CDC7, CDK10, CDK12, CDK13, CDK14, CDK15, CDK16, CDK17, CDK18, CDK19, CDK1, CDK20, CDK2, CDK3, CDK4, CDK5, CDK6, CDK7, CDK8, CDK9, CDKL1, CDKL2, CDKL3, CDKL4, CDKL5, CHEK1, CHEK2, CHKA, CHKB, CLK1, CLK2, CLK3, CLK4, COQ8A, COQ8B, CSF1R, CSNK2A1, CSNK2A2, CSNK2A3, CSK, CASK, CIT, DAPK1, DAPK2, DAPK3, DCLK1, DCLK2, DCLK3, DDR1, DDR2, DIPK1C, DMPK, DSTYK, DYRK1A, DYRK1B, DYRK2, DYRK3, DYRK4, EIF2AK1, EIF2AK2, EIF2AK3, EIF2AK4, EEF2K, EGFR, ETNK1, ETNK2, EPHA1, EPHA2, EPHA3, EPHA4, EPHA5, EPHA6, EPHA7, EPHA8, EPHA10, EPHB1, EPHB2, EPHB3, EP

How many statements did we get?

In [12]:
len(kin_stmts)

1042

## Assembling statements and summing evidence and source counts

Before we assemble, we should take note of statements that now have the same hash (i.e., same top-level structure) but different UUIDs (i.e., were originally distinct statements) due to the re-grounding that we did. We build a mapping dict so that we can subsequently add the evidence and source counts together for the statements that now overlap:

In [13]:
hash_to_uuids = {}
for stmt in kin_stmts:
    if stmt.get_hash(refresh=True) not in hash_to_uuids:
        hash_to_uuids[stmt.get_hash(refresh=True)] = []
    hash_to_uuids[stmt.get_hash(refresh=True)].append(stmt.uuid)

Now we use the INDRA Preassembler to combine evidences from statements that are now duplicates.

In [14]:
pa = Preassembler(None)
kin_stmts = pa.combine_duplicate_stmts(kin_stmts)
len(kin_stmts)

911

Because we will want to rank kinases by their overall evidence counts, we get the total evidence counts for each unique statement (determined by unique hash) by summing evidence counts from the originally distinct statements (stored by UUID):

In [15]:
# Now add up the evidence and source counts after assembly
final_ev_counts = {}
final_src_counts = {}

# Add up the evidences across the UUIDs for the statement
for stmt in kin_stmts:
    new_hash = stmt.get_hash(refresh=True)
    # Initialize the dict for this statement hash
    if new_hash not in final_ev_counts:
        final_ev_counts[new_hash] = 0
    if new_hash not in final_src_counts:
        final_src_counts[new_hash] = {}
    # Get all the previously distinct statements by UUID and add up
    # their evidence and source counts
    for uuid in hash_to_uuids[new_hash]:
        final_ev_counts[new_hash] += ev_counts_by_uuid[uuid] 
        for src_name, src_ct in src_counts_by_uuid[uuid].items():
            # Initialize the count for this source to 0
            if src_name not in final_src_counts[new_hash]:
                final_src_counts[new_hash][src_name] = 0
            # Add up all evidences for this source
            final_src_counts[new_hash][src_name] += src_ct

## Group by identifier type (gene or family) and save

Sort the statements into those that have a human gene as subject, a human protein family as subject, or neither (which we exclude).

In [16]:
hgnc_stmts = [] # HGNC kinase statements
fplx_stmts = [] # FPLX kinase statements
not_hgnc_fplx_stmts = [] # Statements with neither HGNC nor FPLX (only 2 of these)

for stmt in kin_stmts:
    # Get the identifiers for the subject/kinase agent
    ag_db_refs = stmt.agent_list()[0].db_refs
    # Check if the subject agent has an HGNC identifier
    if 'HGNC' in ag_db_refs:
        hgnc_stmts.append(stmt)
    # Check if it has a FPLX identifier (indicating a protein family)
    elif 'FPLX' in ag_db_refs:
        fplx_stmts.append(stmt)
    # Statement has neither HGNC or FPLX IDs
    else:
        not_hgnc_fplx_stmts = []

Save the statements as JSON:

In [17]:
with open('kinases_motility_hgnc.json', 'w') as f:
    json.dump(stmts_to_json(hgnc_stmts), f)
    
with open('kinases_motility_fplx.json', 'w') as f:
    json.dump(stmts_to_json(fplx_stmts), f)

## How many kinases do we find?

How many kinases do we find to be linked to motility?

In [18]:
hgnc_kin = set([stmt.agent_list()[0].name for stmt in kin_stmts
                if 'HGNC' in stmt.agent_list()[0].db_refs])
fplx_kin = set([stmt.agent_list()[0].name for stmt in kin_stmts
                if 'FPLX' in stmt.agent_list()[0].db_refs])

How many distinct genes linked to motility?

In [19]:
len(hgnc_kin)

313

How many protein families?

In [20]:
len(fplx_kin)

44

Names of the families:

In [21]:
fplx_kin

{'ABL_family',
 'AKT',
 'AMPK',
 'AMPK_alpha',
 'CAMK',
 'CAMK2_complex',
 'CDK',
 'CK2',
 'CSNK1',
 'DAPK',
 'DDR',
 'EPHB',
 'ERBB',
 'ERK',
 'Ephrin_receptor',
 'FGFR',
 'GSK3',
 'IKK_complex',
 'JAK',
 'JNK',
 'MAPK',
 'MEK',
 'MKNK',
 'Myosin_complex',
 'PAK',
 'PDGFR',
 'PI3K',
 'PI3K_p110',
 'PIM',
 'PKA',
 'PKC',
 'PRKG',
 'RAF',
 'ROCK',
 'ROR',
 'RSK',
 'RSTK1',
 'SRC',
 'TAT_associated_kinase',
 'TGFBR',
 'VEGFR',
 'mTORC1',
 'mTORC2',
 'p38'}

## Make a tabular dump of summary information

For handy reference, we make a tabular view of the HGNC and FPLX statements indicating

1. the kinase
2. Identifier namespace for the kinase (HGNC or FPLX)
3. Identifier (HGNC ID or FPLX ID, which is same as the FPLX name)
4. the number of total sentences for both "cell motility" and "cell migration"
5. the number of sentences for positive regulation of motility/migration
6. the number of sentences for negative regulation of motility/migration
7. a link out to results on INDRA DB for regulation of cell motility
8. a link out to results on INDRA DB for regulation of cell migration


In [22]:
def get_ev_counts_by_kinase(stmts, db_ns):
    counts = {}
    for stmt in stmts:
        kin_name = stmt.agent_list()[0].name
        db_id = stmt.agent_list()[0].db_refs[db_ns]
        stmt_ev_ct = final_ev_counts[stmt.get_hash(refresh=True)]
        motility_link = 'https://db.indra.bio/statements/from_agents?' \
                        f'subject={db_id}@{db_ns}&object=GO:0048870@GO&format=html'
        migration_link = 'https://db.indra.bio/statements/from_agents?' \
                        f'subject={db_id}@{db_ns}&object=GO:0016477@GO&format=html'
        if kin_name not in counts:
            counts[kin_name] = {'total_evidence': 0, 'pos_reg_evidence': 0,
                                'neg_reg_evidence': 0, 'db_ns': db_ns,
                                'db_id': db_id,
                                'motility_link': motility_link,
                                'migration_link': migration_link}
        counts[kin_name]['total_evidence'] += stmt_ev_ct
        if isinstance(stmt, Activation):
            counts[kin_name]['pos_reg_evidence'] += stmt_ev_ct
        elif isinstance(stmt, Inhibition):
            counts[kin_name]['neg_reg_evidence'] += stmt_ev_ct
    return counts

hgnc_counts = get_ev_counts_by_kinase(hgnc_stmts, 'HGNC')
fplx_counts = get_ev_counts_by_kinase(fplx_stmts, 'FPLX')

Write the results summaries to CSV files:

In [23]:
# Write the summary results to CSV files
def dump_summary_csv(counts_dict, filename):
    # CSV Header
    header = [['NAME', 'DB_NAMESPACE', 'DB_ID', 'TOTAL_EVIDENCE',
             'POS_REG_EVIDENCE', 'NEG_REG_EVIDENCE',
             'MOTILITY_LINK', 'MIGRATION_LINK']]
    data = [(kin_name, kin_dict['db_ns'], kin_dict['db_id'],
              kin_dict['total_evidence'],
             kin_dict['pos_reg_evidence'], kin_dict['neg_reg_evidence'],
             kin_dict['motility_link'], kin_dict['migration_link'])
            for kin_name, kin_dict in counts_dict.items()]
    # Sort by total evidence count
    data.sort(key=lambda x: x[3], reverse=True)
    with open(filename, 'w') as f:
        csvwriter = csv.writer(f, delimiter=',')
        csvwriter.writerows(header + data)

dump_summary_csv(hgnc_counts, 'kinases_motility_hgnc_summary.csv')
dump_summary_csv(fplx_counts, 'kinases_motility_fplx_summary.csv')

## Make HTML view of statements

To browse the text evidence linking each kinase to cell motility/migration, we generate an HTML view. We want to sort the results so the kinases with greater evidence of being link to either migration or motility show up at the top. To do this, we first total the evidences across all statements involving the same kinase.

In [24]:
# For sorting purposes, we add up all the evidences for the subject kinase
# across both GO terms
ev_by_subj = {}
for stmt in kin_stmts:
    stmt_subj = stmt.agent_list()[0].name    
    if stmt_subj not in ev_by_subj:
        ev_by_subj[stmt_subj] = 0
    ev_by_subj[stmt_subj] += final_ev_counts[stmt.get_hash()]

# We then apply these evidence totals to each individual statement
subj_ev_by_hash = {}
for stmt in kin_stmts:
    stmt_subj = stmt.agent_list()[0].name
    subj_ev = ev_by_subj[stmt_subj]
    subj_ev_by_hash[stmt.get_hash()] = subj_ev

Because this is an unusual type of sorting (grouping evidence counts by subject rather than by statement) we create a custom StmtStat object and sorting function to pass to the HTML Assembler:

In [25]:
# This aggregator passes the counts at the statement level to the statement
# group level without doing anything (no adding, averaging, etc.)
class EmptyAggregator(sp.BasicAggregator):
    def _merge(self, metric_array):
        self._values = metric_array

subj_ev_stat = sp.StmtStat('subject_ev', subj_ev_by_hash, int, EmptyAggregator)

# We sort by the subject evidence count first, and secondarily by the
# statement-level evidence count.
def sort_func(metrics):
    return (metrics['subject_ev'], metrics['ev_count'])

Initialize the HtmlAssembler object that will generate the HTML, using our assembled evidence and source counts, and custom subject-level evidence total, and sort function: 

In [26]:
ha = HtmlAssembler(kin_stmts, ev_counts=final_ev_counts, source_counts=final_src_counts,
                   title='Kinases regulating cell motility and migration', db_rest_url='https://db.indra.bio',
                   custom_stats=[subj_ev_stat], sort_by=sort_func)

In [27]:
ha.make_model(grouping_level='agent-pair')
ha.save_model('kinases_motility.html')

INFO: [2021-01-15 09:52:37] indra.assemblers.html.assembler - Removing NCIT from refs due to too many matches: {'C16376', 'C26360'}
INFO: [2021-01-15 09:52:37] indra.assemblers.html.assembler - Removing MESH from refs due to too many matches: {'D020928', 'D048049'}
INFO: [2021-01-15 09:52:37] indra.assemblers.html.assembler - Removing NCIT from refs due to too many matches: {'C16376', 'C26360'}
INFO: [2021-01-15 09:52:37] indra.assemblers.html.assembler - Removing MESH from refs due to too many matches: {'C000628936', 'D019009'}
INFO: [2021-01-15 09:52:37] indra.assemblers.html.assembler - Removing UP from refs due to too many matches: {'Q05438', 'O00238'}
